<a href="https://colab.research.google.com/github/seadreamjj/danmu-ID/blob/main/%E7%8C%AB%E8%80%B3%E5%BC%B9%E5%B9%95id%E8%AE%A1%E7%AE%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

猫耳付费id抓取，正确版


In [ ]:
import requests
import json
import re
import pandas as pd

from datetime import datetime
import pytz

# 获取本地时区的当前时间
local_time = datetime.now(pytz.timezone('Europe/Stockholm'))  # 这里你可以指定你需要的时区

# 打印本地时区时间
formatted_time = local_time.strftime("%Y-%m-%d %H:%M:%S")
print("时间:", formatted_time)


# 输入剧集地址
drama_id = 'https://www.missevan.com/mdrama/71233'

# 构建 URL
drama_url = 'http://www.missevan.com/dramaapi/getdrama?drama_id='
url = drama_url + drama_id.split("/")[4]

# 请求剧集数据
drama_result = requests.get(url).text

#获取剧集名称
drama_data = json.loads(drama_result)
drama_name = drama_data["info"]["drama"]["name"]

# 使用正则表达式提取 sound_id 和 pay_type
pattern1 = re.compile(r'"sound_id":(\d+),')  # 提取 sound_id
pattern2 = re.compile(r'"need_pay":(\d+),')  # 提取付费状态 need_pay
sound_id = re.findall(pattern1, drama_result)
pay_type = re.findall(pattern2, drama_result)

# 去掉第一个非付费项
pay_type = pay_type[1:]

# 创建 DataFrame
drama = pd.DataFrame({'sound_id': sound_id, 'pay_type': pay_type})

# 筛选付费 sound_id
sound_id = list(drama['sound_id'][drama['pay_type'] == '1'])

sound_url = 'http://www.missevan.com/sound/getdm?soundid='
dm1 = pd.DataFrame()

# 循环处理每个 sound_id，获取弹幕信息
for i in range(len(sound_id)):
    url = sound_url + sound_id[i]
    sound_result = requests.get(url).text

    # 提取弹幕内容
    pattern3 = re.compile(r'p="(.+?)"')
    dm = re.findall(pattern3, sound_result)

    # 将弹幕数据存储到 DataFrame
    dm_temp = pd.DataFrame({'dm': dm})
    dms_split = dm_temp['dm'].str.split('>', expand=True)[0]
    uid = dms_split.str.split(',', expand=True)[6]
    dm_temp['uid'] = uid

    # 合并数据，使用 pd.concat() 替代 append()
    if i == 0:
        dm1 = dm_temp
    else:
        dm1 = pd.concat([dm1, dm_temp], ignore_index=True)

# 输出不重复的 UID 数量和总弹幕数量

print(f"{drama_name} 实时付费ID为：{len(dm1['uid'].unique())}")

print("本剧实时付费弹幕数量为：", len(dm1['uid']))


时间: 2024-11-08 17:27:24
City of Angels 全一季 实时付费ID为：5398
本剧实时付费弹幕数量为： 108043


# 新段落

漫播弹幕id抓取


In [ ]:
import requests
import re
import json

from datetime import datetime
import pytz

# 获取本地时区的当前时间
local_time = datetime.now(pytz.timezone('Europe/Stockholm'))  # 这里你可以指定你需要的时区

# 打印本地时区时间
formatted_time = local_time.strftime("%Y-%m-%d %H:%M:%S")
print("系统本地时间:", formatted_time)


# 输入漫播剧集的任意一集链接（PC 或 APP 端）
url = "https://manbo.hongdoulive.com/Activecard/episode?id=2004479649836433539&radioDramaId=1987053485752844488&t=1730806200107&share_tpye=h5&fr=2"

# 判断链接类型
if 'pc' in url:
    detail_id = url.split('Id=')[1].split('&')[0]  # 提取 detail_id
    new_url = "https://manbo.hongdoulive.com/web_manbo/dramaSetDetail?dramaSetId=" + detail_id
    temp = 'data'
else:
    episode_id = url.split('id=')[1][:19]  # 提取 episode_id
    radio_id = url.split('DramaId=')[1][:19]  # 提取 radio_id
    new_url = "https://manbo.hongrenshuo.com.cn/api/v207/radio/drama/set/h5/detail?radioDramaSetId=%s&radioDramaId=%s" % (episode_id, radio_id)
    temp = 'b'


r= requests.get(new_url).text
r= str(json.loads(r)[temp]['radioDramaResp']['setRespList'])
# 请求新的 URL，获取剧集信息
#response = requests.get(new_url).text

# 解析 JSON 数据，提取付费集信息
#r = json.loads(response)
pattern1 = re.compile("'vipFree': (.+?),")
vipFree = re.findall(pattern1, r)
num = len(vipFree)

pattern2 = re.compile("'setId': (.+?),")
setId = re.findall(pattern2, r)

# 找到所有免费集或付费集的 ID
episodes = [setId[i] for i in range(num) if vipFree[i] == '1']

# 如果没有免费集，查找付费集
if len(episodes) == 0:
    pattern3 = re.compile("'payType': (.+?),")
    payType = re.findall(pattern3, r)
    episodes = [setId[i] for i in range(num) if payType[i] == '1']

# 弹幕列表
danmu = []

# 获取每集的弹幕信息
for each in episodes:
    next_url = "https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=%s&startTime=0&endTime=10000000" % each
    print(next_url)
    danmakuList = json.loads(requests.get(next_url).content)['b']['danmakuList']

    # 提取每条弹幕的 eid
    for j in danmakuList:
        danmu.append(j['eid'])

# 输出不重复的弹幕 ID 数量和总弹幕数量
print("付费集ID数：%d" % len(set(danmu)))
print("付费集弹幕数：%d" % len(danmu))


系统本地时间: 2024-11-08 10:55:37
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=2012307159013916702&startTime=0&endTime=10000000
付费集ID数：93
付费集弹幕数：418


试图计算猫耳收费剧集的平均时长


In [ ]:
import requests
import re
import pandas as pd

# 输入剧集地址
drama_id = 'https://www.missevan.com/mdrama/77685'

# 构建 URL
drama_url = 'http://www.missevan.com/dramaapi/getdrama?drama_id='
url = drama_url + drama_id.split("/")[4]

# 请求剧集数据
drama_result = requests.get(url).text

# 使用正则表达式提取 sound_id、pay_type 和 duration
pattern1 = re.compile(r'"sound_id":(\d+),')  # 提取 sound_id
pattern2 = re.compile(r'"need_pay":(\d+),')  # 提取付费状态 need_pay
pattern3 = re.compile(r'"duration":(\d+),')  # 提取时长 duration (秒)

sound_id = re.findall(pattern1, drama_result)
pay_type = re.findall(pattern2, drama_result)
durations = re.findall(pattern3, drama_result)

# 打印各列表的长度，检查是否一致
print("sound_id 长度:", len(sound_id))
print("pay_type 长度:", len(pay_type))
print("durations 长度:", len(durations))

# 去掉第一个非付费项（通常第一个为非付费）
pay_type = pay_type[1:]
durations = durations[1:]

# 确保长度一致后才能继续创建 DataFrame
min_length = min(len(sound_id), len(pay_type), len(durations))

# 截断为相同的长度
sound_id = sound_id[:min_length]
pay_type = pay_type[:min_length]
durations = durations[:min_length]

# 创建 DataFrame，包含 sound_id、pay_type 和 duration
drama = pd.DataFrame({
    'sound_id': sound_id,
    'pay_type': pay_type,
    'duration': [int(dur) for dur in durations]  # 将时长转换为整数
})

# 筛选付费集
paid_drama = drama[drama['pay_type'] == '1']

# 计算平均时长（秒 -> 分钟）
average_duration_minutes = paid_drama['duration'].mean() / 60

# 输出收费集的平均时长（分钟）
print("收费集的平均时长为：%.2f 分钟" % average_duration_minutes)


sound_id 长度: 26
pay_type 长度: 27
durations 长度: 26
收费集的平均时长为：21307.96 分钟


算平均时长

In [ ]:
import requests
import re
import pandas as pd

# 输入猫耳剧集的 URL
drama_id = 'https://www.missevan.com/mdrama/67153'

# 构建 API URL
drama_url = 'http://www.missevan.com/dramaapi/getdrama?drama_id='
url = drama_url + drama_id.split("/")[4]

# 请求剧集数据
drama_result = requests.get(url).text

# 使用正则表达式提取 sound_id 和 pay_type
pattern1 = re.compile(r'"sound_id":(\d+),')  # 提取 sound_id
pattern2 = re.compile(r'"need_pay":(\d+),')  # 提取付费状态 need_pay
sound_id = re.findall(pattern1, drama_result)
pay_type = re.findall(pattern2, drama_result)

# 去掉第一个非付费项
pay_type = pay_type[1:]

# 创建 DataFrame，筛选付费集
drama = pd.DataFrame({'sound_id': sound_id, 'pay_type': pay_type})
paid_sound_id = list(drama['sound_id'][drama['pay_type'] == '1'])

# 定义获取音频时长的函数
def get_audio_duration(sound_id):
    # 构建 avinfo 请求 URL
    audio_url = f"https://www.missevan.com/sound/getsound?soundid={sound_id}&avinfo"
    response = requests.get(audio_url).json()

    # 检查 'format' 字段是否存在
    if 'format' in response:
        duration = float(response['format'].get('duration', 0))
        return duration / 60  # 返回分钟数
    # 如果 'format' 字段不存在，检查 'streams' 字段
    elif 'streams' in response:
        stream_info = response['streams'][0]  # 通常时长信息在第一个流中
        if 'duration' in stream_info:
            duration = float(stream_info.get('duration', 0))
            return duration / 60  # 返回分钟数
        else:
            raise KeyError("'streams' 字段中缺少时长信息")
    else:
        raise KeyError("'format' 和 'streams' 字段都缺失")

# 获取每个收费音频的时长
durations = []
for sid in paid_sound_id:
    try:
        duration = get_audio_duration(sid)
        durations.append(duration)
        print(f"音频 sound_id: {sid}, 时长: {duration:.2f} 分钟")
    except Exception as e:
        print(f"获取 sound_id {sid} 时长时出错: {e}")

# 计算收费集的平均时长
if durations:
    avg_duration = sum(durations) / len(durations)
    print(f"收费集的平均时长: {avg_duration:.2f} 分钟")
else:
    print("没有获取到收费集的时长信息。")


获取 sound_id 8436631 时长时出错: "'format' 和 'streams' 字段都缺失"
获取 sound_id 8473659 时长时出错: "'format' 和 'streams' 字段都缺失"
获取 sound_id 8511196 时长时出错: "'format' 和 'streams' 字段都缺失"
获取 sound_id 8510859 时长时出错: "'format' 和 'streams' 字段都缺失"
获取 sound_id 8540011 时长时出错: "'format' 和 'streams' 字段都缺失"
获取 sound_id 8572502 时长时出错: "'format' 和 'streams' 字段都缺失"
获取 sound_id 8605095 时长时出错: "'format' 和 'streams' 字段都缺失"
获取 sound_id 8604873 时长时出错: "'format' 和 'streams' 字段都缺失"
获取 sound_id 8637984 时长时出错: "'format' 和 'streams' 字段都缺失"
获取 sound_id 8667760 时长时出错: "'format' 和 'streams' 字段都缺失"
获取 sound_id 8698993 时长时出错: "'format' 和 'streams' 字段都缺失"
获取 sound_id 8703288 时长时出错: "'format' 和 'streams' 字段都缺失"
获取 sound_id 8730391 时长时出错: "'format' 和 'streams' 字段都缺失"
获取 sound_id 8762298 时长时出错: "'format' 和 'streams' 字段都缺失"
获取 sound_id 8792924 时长时出错: "'format' 和 'streams' 字段都缺失"
获取 sound_id 8796330 时长时出错: "'format' 和 'streams' 字段都缺失"
没有获取到收费集的时长信息。


《台风眼》第二季从第二集开始计算ID

In [ ]:
import requests
import re
import pandas as pd

# 输入剧集地址
drama_id = 'https://www.missevan.com/mdrama/68664'

# 构建 URL
drama_url = 'http://www.missevan.com/dramaapi/getdrama?drama_id='
url = drama_url + drama_id.split("/")[4]

# 请求剧集数据
drama_result = requests.get(url).text

# 使用正则表达式提取 sound_id 和 pay_type
pattern1 = re.compile(r'"sound_id":(\d+),')  # 提取 sound_id
pattern2 = re.compile(r'"need_pay":(\d+),')  # 提取付费状态 need_pay
sound_id = re.findall(pattern1, drama_result)
pay_type = re.findall(pattern2, drama_result)

# 去掉第一个非付费项
pay_type = pay_type[1:]

# 创建 DataFrame
drama = pd.DataFrame({'sound_id': sound_id, 'pay_type': pay_type})

# 筛选付费 sound_id
sound_id = list(drama['sound_id'][drama['pay_type'] == '1'])

# 从第二个 sound_id 开始处理
sound_id = sound_id[1:]  # 跳过第一个（即第一集）

sound_url = 'http://www.missevan.com/sound/getdm?soundid='
dm1 = pd.DataFrame()

# 循环处理每个 sound_id，获取弹幕信息
for i in range(len(sound_id)):
    url = sound_url + sound_id[i]
    sound_result = requests.get(url).text

    # 提取弹幕内容
    pattern3 = re.compile(r'p="(.+?)"')
    dm = re.findall(pattern3, sound_result)

    # 将弹幕数据存储到 DataFrame
    dm_temp = pd.DataFrame({'dm': dm})
    dms_split = dm_temp['dm'].str.split('>', expand=True)[0]
    uid = dms_split.str.split(',', expand=True)[6]
    dm_temp['uid'] = uid

    # 合并数据，使用 pd.concat() 替代 append()
    if i == 0:
        dm1 = dm_temp
    else:
        dm1 = pd.concat([dm1, dm_temp], ignore_index=True)

# 输出不重复的 UID 数量和总弹幕数量
print("台风眼2去除第一集数据后实时付费ID为：", len(dm1['uid'].unique()))
print("本剧实时付费弹幕数量为：", len(dm1['uid']))


台风眼2去除第一集数据后实时付费ID为： 4813
本剧实时付费弹幕数量为： 141938


按时间过滤

In [ ]:
import requests
import re
import pandas as pd
from datetime import datetime, timezone, timedelta

# 输入剧集地址
drama_id = 'https://www.missevan.com/mdrama/68664'

# 构建 URL
drama_url = 'http://www.missevan.com/dramaapi/getdrama?drama_id='
url = drama_url + drama_id.split("/")[4]

# 请求剧集数据
drama_result = requests.get(url).text

# 使用正则表达式提取 sound_id 和 pay_type
pattern1 = re.compile(r'"sound_id":(\d+),')  # 提取 sound_id
pattern2 = re.compile(r'"need_pay":(\d+),')  # 提取付费状态 need_pay
sound_id = re.findall(pattern1, drama_result)
pay_type = re.findall(pattern2, drama_result)

# 去掉第一个非付费项
pay_type = pay_type[1:]

# 创建 DataFrame
drama = pd.DataFrame({'sound_id': sound_id, 'pay_type': pay_type})

# 筛选付费 sound_id
sound_id = list(drama['sound_id'][drama['pay_type'] == '1'])

sound_url = 'http://www.missevan.com/sound/getdm?soundid='
dm1 = pd.DataFrame()

# 循环处理每个 sound_id，获取弹幕信息
for i in range(len(sound_id)):
    url = sound_url + sound_id[i]
    sound_result = requests.get(url).text

    # 提取弹幕内容
    pattern3 = re.compile(r'p="(.+?)"')
    dm = re.findall(pattern3, sound_result)

    # 将弹幕数据存储到 DataFrame
    dm_temp = pd.DataFrame({'dm': dm})
    dms_split = dm_temp['dm'].str.split('>', expand=True)[0]
    uid = dms_split.str.split(',', expand=True)[6]
    send_time = dms_split.str.split(',', expand=True)[4]  # 获取发送时间（时间戳）

    # 添加用户 ID 和发送时间到 DataFrame
    dm_temp['uid'] = uid
    dm_temp['send_time'] = send_time

    # 合并数据，使用 pd.concat() 替代 append()
    if i == 0:
        dm1 = dm_temp
    else:
        dm1 = pd.concat([dm1, dm_temp], ignore_index=True)

# 设置时间阈值（2024年7月3日13点12分）
threshold_time = datetime(2024, 7, 3, 13, 12, 0, tzinfo=timezone.utc).timestamp()

# 提取在指定时间之后的 uid
valid_uids = set()

for index, row in dm1.iterrows():
    uid = row['uid']
    timestamp = int(row['send_time'])

    # 检查发送时间是否在阈值之后
    if timestamp >= threshold_time:
        valid_uids.add(uid)

# 输出去重后的用户 ID 数量
print("台风眼2在2024年7月3日13点12分以后的弹幕用户 ID 去重数量:", len(valid_uids))

台风眼2在2024年7月3日13点12分以后的弹幕用户 ID 去重数量: 4838


In [ ]:
试试漫播有声剧

In [ ]:
from datetime import datetime
import requests
import re
import json

# 获取当前时间
current_time = datetime.now()

# 打印当前时间
print("当前时间:", current_time)


# 输入漫播剧集的任意一集链接（PC 或 APP 端）
url = "https://www.kilamanbo.com/manbo/pc/detail?id=1944764730950287431&collectId=1966909818551664738"

# 判断是否为 PC 端链接
if 'pc' in url:
    detail_id = url.split('Id=')[1].split('&')[0]  # 提取 detail_id
    new_url = "https://manbo.hongdoulive.com/web_manbo/dramaSetDetail?dramaSetId=" + detail_id
    temp = 'data'
else:
    try:
        # 提取 APP 端的 episode_id 和 radio_id
        episode_id = re.search(r'id=(\d+)', url).group(1)  # 提取 episode_id
        radio_id = re.search(r'radioDramaId=(\d+)', url).group(1)  # 提取 radioDramaId
        new_url = "https://manbo.hongrenshuo.com.cn/api/v207/radio/drama/set/h5/detail?radioDramaSetId=%s&radioDramaId=%s" % (episode_id, radio_id)
        temp = 'b'
    except Exception as e:
        print(f"APP端链接解析错误: {e}")

r= requests.get(new_url).text
r= str(json.loads(r)[temp]['radioDramaResp']['setRespList'])
# 请求新的 URL，获取剧集信息
#response = requests.get(new_url).text

# 解析 JSON 数据，提取付费集信息
#r = json.loads(response)
# 使用正则表达式提取 vipFree, payType, setId 和 price
pattern_vipFree = re.compile("'vipFree': (.+?),")
pattern_payType = re.compile("'payType': (.+?),")
pattern_setId = re.compile("'setId': (.+?),")
pattern_price = re.compile("'price': (.+?),")

vipFree = re.findall(pattern_vipFree, r)
payType = re.findall(pattern_payType, r)
setId = re.findall(pattern_setId, r)
price = re.findall(pattern_price, r)

# 过滤掉非有效数据项，确保长度一致
num = len(vipFree)


# 找到所有免费集或付费集的 ID
episodes = [setId[i] for i in range(num) if vipFree[i] == '1']

# 如果没有免费集，查找付费集
if len(episodes) == 0:
    pattern3 = re.compile("'payType': (.+?),")
    payType = re.findall(pattern3, r)
    episodes = [setId[i] for i in range(num) if payType[i] == '1']

#如果都没有，检查是否按集付费
if len(episodes) == 0:
    episodes = [setId[i] for i in range(num) if float(price[i]) > 0]

# 打印符合条件的集数 ID
print(f"符合条件的剧集ID: {episodes}")
# 弹幕列表
danmu = []

# 获取每集的弹幕信息
for each in episodes:
    next_url = "https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=%s&startTime=0&endTime=10000000" % each
    print(next_url)
    danmakuList = json.loads(requests.get(next_url).content)['b']['danmakuList']

    # 提取每条弹幕的 eid
    for j in danmakuList:
        danmu.append(j['eid'])

# 输出不重复的弹幕 ID 数量和总弹幕数量
print("付费集ID数：%d" % len(set(danmu)))
print("付费集弹幕数：%d" % len(danmu))


当前时间: 2024-10-28 21:19:46.709605
符合条件的剧集ID: ['1972503008432357537', '1975132550619725955', '1977715801649578080', '1980245223854309438', '1982901424903684266', '1985499098433716288', '1988100293836931134', '1990677854035116060', '1993256900291985543', '1995885304313020545', '1998492993706786915', '2001089404516434155', '2003691166855332006', '2006282668517359693', '1992369426904645730', '1997636186385940513', '2002920482218704958', '2008131935406129286', '1981171382012084546', '1981171390602019076', '2008889163910217945']
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1972503008432357537&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1975132550619725955&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1977715801649578080&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/d

In [ ]:
# 输入漫播剧集的任意一集链接（PC 或 APP 端）
url = "https://www.kilamanbo.com/manbo/pc/detail?id=1966611752116289550&collectId=1972913680320299077"

# 判断是否为 PC 端链接
if 'pc' in url:
    detail_id = url.split('Id=')[1].split('&')[0]  # 提取 detail_id
    new_url = "https://manbo.hongdoulive.com/web_manbo/dramaSetDetail?dramaSetId=" + detail_id
    temp = 'data'
else:
    try:
        # 提取 APP 端的 episode_id 和 radio_id
        episode_id = re.search(r'id=(\d+)', url).group(1)  # 提取 episode_id
        radio_id = re.search(r'radioDramaId=(\d+)', url).group(1)  # 提取 radioDramaId
        new_url = "https://manbo.hongrenshuo.com.cn/api/v207/radio/drama/set/h5/detail?radioDramaSetId=%s&radioDramaId=%s" % (episode_id, radio_id)
        temp = 'b'
    except Exception as e:
        print(f"APP端链接解析错误: {e}")

# 打印结果
print(f"生成的 URL: {new_url}")
print(f"提取结果: {temp}")


生成的 URL: https://manbo.hongdoulive.com/web_manbo/dramaSetDetail?dramaSetId=1972913680320299077
提取结果: data


把漫播弹幕数据分类输出

In [ ]:
import requests
import re
import json

# 输入漫播剧集的任意一集链接（PC 或 APP 端）
url = "https://www.kilamanbo.com/manbo/pc/detail?id=1944764730950287431&collectId=1966909818551664738"

# 判断链接类型
if 'pc' in url:
    detail_id = url.split('Id=')[1].split('&')[0]  # 提取 detail_id
    new_url = "https://manbo.hongdoulive.com/web_manbo/dramaSetDetail?dramaSetId=" + detail_id
    temp = 'data'
else:
    episode_id = url.split('id=')[1][:19]  # 提取 episode_id
    radio_id = url.split('DramaId=')[1][:19]  # 提取 radio_id
    new_url = "https://manbo.hongrenshuo.com.cn/api/v207/radio/drama/set/h5/detail?radioDramaSetId=%s&radioDramaId=%s" % (episode_id, radio_id)
    temp = 'b'

r = requests.get(new_url).text
r = str(json.loads(r)[temp]['radioDramaResp']['setRespList'])

# 解析付费集信息
pattern1 = re.compile("'vipFree': (.+?),")
vipFree = re.findall(pattern1, r)
num = len(vipFree)

pattern2 = re.compile("'setId': (.+?),")
setId = re.findall(pattern2, r)

# 找到所有免费集或付费集的 ID
episodes = [setId[i] for i in range(num) if vipFree[i] == '1']

# 如果没有免费集，查找付费集
if len(episodes) == 0:
    pattern3 = re.compile("'payType': (.+?),")
    payType = re.findall(pattern3, r)
    episodes = [setId[i] for i in range(num) if payType[i] == '1']

# 弹幕分类列表
danmu_source_1 = []  # 弹幕数据
danmu_source_2 = []  # 评论数据
danmu_source_3 = []  # +1 数据
danmu_source_4 = []
danmu_source_5 = []
danmu_source_6 = []

# 获取每集的弹幕信息
for each in episodes:
    next_url = "https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=%s&startTime=0&endTime=10000000" % each
    print(next_url)
    danmakuList = json.loads(requests.get(next_url).content)['b']['danmakuList']

    # 根据 danmakuSource 分类弹幕
    for j in danmakuList:
        danmaku_source = j.get('danmakuSource')
        eid = j.get('eid')

        if danmaku_source == 1:
            danmu_source_1.append(eid)
        elif danmaku_source == 2:
            danmu_source_2.append(eid)
        elif danmaku_source == 3:
            danmu_source_3.append(eid)
        elif danmaku_source == 4:
            danmu_source_4.append(eid)
        elif danmaku_source == 5:
            danmu_source_5.append(eid)
        elif danmaku_source == 6:
            danmu_source_6.append(eid)

# 输出每类弹幕的 ID 数量和总数量（针对付费集）
print("付费集弹幕ID数 (danmakuSource = 1)：%d" % len(set(danmu_source_1)))
print("付费集评论ID数 (danmakuSource = 2)：%d" % len(set(danmu_source_2)))
print("付费集+1 ID数量 (danmakuSource = 3)：%d" % len(set(danmu_source_3)))
print(" ID数量 (danmakuSource = 4)：%d" % len(set(danmu_source_4)))
print(" ID数量 (danmakuSource = 5)：%d" % len(set(danmu_source_5)))
print(" ID数量 (danmakuSource = 6)：%d" % len(set(danmu_source_6)))


print("付费集发弹幕产生的弹幕总数 (danmakuSource = 1)：%d" % len(danmu_source_1))
print("付费集评论产生的弹幕总数 (danmakuSource = 2)：%d" % len(danmu_source_2))
print("付费集+1 产生的弹幕总数 (danmakuSource = 3)：%d" % len(danmu_source_3))
print("source4弹幕总数 (danmakuSource = 4)：%d" % len(danmu_source_4))
print("source5弹幕总数 (danmakuSource = 5)：%d" % len(danmu_source_5))
print("source6弹幕总数 (danmakuSource = 6)：%d" % len(danmu_source_6))



https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1972503008432357537&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1975132550619725955&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1977715801649578080&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1980245223854309438&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1982901424903684266&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1985499098433716288&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1988100293836931134&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/a

只输出总ID和弹幕ID

In [ ]:
import requests
import re
import json

# 输入漫播剧集的任意一集链接（PC 或 APP 端）
url = "https://www.kilamanbo.com/manbo/pc/detail?id=1966611752116289550&collectId=1972913680320299077"

# 判断链接类型
if 'pc' in url:
    detail_id = url.split('Id=')[1].split('&')[0]  # 提取 detail_id
    new_url = "https://manbo.hongdoulive.com/web_manbo/dramaSetDetail?dramaSetId=" + detail_id
    temp = 'data'
else:
    episode_id = url.split('id=')[1][:19]  # 提取 episode_id
    radio_id = url.split('DramaId=')[1][:19]  # 提取 radio_id
    new_url = "https://manbo.hongrenshuo.com.cn/api/v207/radio/drama/set/h5/detail?radioDramaSetId=%s&radioDramaId=%s" % (episode_id, radio_id)
    temp = 'b'

r = requests.get(new_url).text
r = str(json.loads(r)[temp]['radioDramaResp']['setRespList'])

# 解析付费集信息
pattern1 = re.compile("'vipFree': (.+?),")
vipFree = re.findall(pattern1, r)
num = len(vipFree)

pattern2 = re.compile("'setId': (.+?),")
setId = re.findall(pattern2, r)

# 找到所有付费集的 ID
episodes = [setId[i] for i in range(num) if vipFree[i] == '1']  # 筛选出免费集
# 如果没有免费集，则查找付费集
if len(episodes) == 0:
    pattern3 = re.compile("'payType': (.+?),")
    payType = re.findall(pattern3, r)
    episodes = [setId[i] for i in range(num) if payType[i] == '1']  # 筛选出付费集

# 弹幕分类列表
all_danmu = []         # 所有类型弹幕 ID
danmu_source_1 = []    # 仅包含 danmakuSource = 1 的弹幕 ID

# 获取每集的弹幕信息
for each in episodes:
    next_url = "https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=%s&startTime=0&endTime=10000000" % each
    print(next_url)
    danmakuList = json.loads(requests.get(next_url).content)['b']['danmakuList']

    # 分类并统计弹幕
    for j in danmakuList:
        danmaku_source = j.get('danmakuSource')
        eid = j.get('eid')

        # 统计所有弹幕 ID
        all_danmu.append(eid)

        # 只统计 danmakuSource = 1 的弹幕 ID
        if danmaku_source == 1:
            danmu_source_1.append(eid)

# 输出结果
print("付费集的弹幕ID总数：%d" % len(set(all_danmu)))
print("去掉评论弹幕和+1弹幕后的ID总数：%d" % len(set(danmu_source_1)))


https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1972918799921315869&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1975815806902075439&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1978103169044971698&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1980712095389319405&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1983328559401271385&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1985931047589642289&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1985932662497345688&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/a

只输出url

In [ ]:
import requests
import re

# 输入剧集地址
drama_id = 'https://www.missevan.com/mdrama/47364'

# 构建 URL
drama_url = 'http://www.missevan.com/dramaapi/getdrama?drama_id='
url = drama_url + drama_id.split("/")[4]

# 请求剧集数据
drama_result = requests.get(url).text

# 使用正则表达式提取 sound_id 和 pay_type
pattern1 = re.compile(r'"sound_id":(\d+),')  # 提取 sound_id
pattern2 = re.compile(r'"need_pay":(\d+),')  # 提取付费状态 need_pay
sound_id = re.findall(pattern1, drama_result)
pay_type = re.findall(pattern2, drama_result)

# 去掉第一个非付费项
pay_type = pay_type[1:]

# 筛选付费 sound_id
paid_sound_ids = [sound_id[i] for i in range(len(pay_type)) if pay_type[i] == '1']

# 构建每个 sound_id 对应的 URL 并打印
sound_url = 'http://www.missevan.com/sound/getdm?soundid='
for sid in paid_sound_ids:
    print(sound_url + sid)


http://www.missevan.com/sound/getdm?soundid=5197137
http://www.missevan.com/sound/getdm?soundid=5252307
http://www.missevan.com/sound/getdm?soundid=5266891
http://www.missevan.com/sound/getdm?soundid=5303704
http://www.missevan.com/sound/getdm?soundid=5359029
http://www.missevan.com/sound/getdm?soundid=5407647
http://www.missevan.com/sound/getdm?soundid=5462118
http://www.missevan.com/sound/getdm?soundid=5513497
http://www.missevan.com/sound/getdm?soundid=5569572
http://www.missevan.com/sound/getdm?soundid=5623241
http://www.missevan.com/sound/getdm?soundid=5677465
http://www.missevan.com/sound/getdm?soundid=5734082
http://www.missevan.com/sound/getdm?soundid=5791398
http://www.missevan.com/sound/getdm?soundid=5840069
http://www.missevan.com/sound/getdm?soundid=5851553
http://www.missevan.com/sound/getdm?soundid=5908318
http://www.missevan.com/sound/getdm?soundid=5964950
http://www.missevan.com/sound/getdm?soundid=6015778
http://www.missevan.com/sound/getdm?soundid=6544943


剧集url

In [ ]:
# 输入剧集ID地址
drama_id = 'https://www.missevan.com/mdrama/47364'

# 构建并输出实际剧集数据接口 URL
drama_url = 'http://www.missevan.com/dramaapi/getdrama?drama_id=' + drama_id.split("/")[4]
print(drama_url)



http://www.missevan.com/dramaapi/getdrama?drama_id=47364


漫播弹幕第四种？

In [ ]:
import requests
import re
import json

# 输入漫播剧集的任意一集链接（PC 或 APP 端）
url = "https://www.kilamanbo.com/manbo/pc/detail?id=1944764730950287431&collectId=1966909818551664738"

# 判断链接类型
if 'pc' in url:
    detail_id = url.split('Id=')[1].split('&')[0]  # 提取 detail_id
    new_url = "https://manbo.hongdoulive.com/web_manbo/dramaSetDetail?dramaSetId=" + detail_id
    temp = 'data'
else:
    episode_id = url.split('id=')[1][:19]  # 提取 episode_id
    radio_id = url.split('DramaId=')[1][:19]  # 提取 radio_id
    new_url = "https://manbo.hongrenshuo.com.cn/api/v207/radio/drama/set/h5/detail?radioDramaSetId=%s&radioDramaId=%s" % (episode_id, radio_id)
    temp = 'b'

r = requests.get(new_url).text
r = str(json.loads(r)[temp]['radioDramaResp']['setRespList'])

# 解析付费集信息
pattern1 = re.compile("'vipFree': (.+?),")
vipFree = re.findall(pattern1, r)
num = len(vipFree)

pattern2 = re.compile("'setId': (.+?),")
setId = re.findall(pattern2, r)

# 找到所有付费集的 ID
episodes = [setId[i] for i in range(num) if vipFree[i] == '1']  # 筛选出免费集
# 如果没有免费集，则查找付费集
if len(episodes) == 0:
    pattern3 = re.compile("'payType': (.+?),")
    payType = re.findall(pattern3, r)
    episodes = [setId[i] for i in range(num) if payType[i] == '1']  # 筛选出付费集

# 弹幕分类列表
title = "万米"
danmu = []
pure_danmu = []  # 存储纯弹幕（danmakuSource = 1）
danmu_plus2 = []  # 存储弹幕ID和+1ID（danmakuSource = 1 + 2）
danmu_plus3 = []  # 存储弹幕ID和+1ID（danmakuSource = 1 + 3）
danmu_plus4 = []  # 存储弹幕ID和+1ID（danmakuSource = 1 + 4）
danmu_plus5 = []  # 存储弹幕ID和+1ID（danmakuSource = 1 + 5）
danmu_plus6 = []  # 存储弹幕ID和+1ID（danmakuSource = 1 + 6）

# 获取每集的弹幕信息
for each in episodes:
    next_url = "https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=%s&startTime=0&endTime=10000000" % each
    print(next_url)
    danmakuList = json.loads(requests.get(next_url).content)['b']['danmakuList']

    # 分类并统计弹幕
    for j in danmakuList:

        eid = j.get('eid')

        # 统计所有弹幕 ID
        danmu.append(eid)

         # 分类存储纯弹幕
        if j.get('danmakuSource') == 1:
           pure_danmu.append(eid)
           danmu_plus2.append(eid)  # 纯弹幕也存入弹幕ID和评论ID列表
           danmu_plus3.append(eid)
           danmu_plus4.append(eid)
           danmu_plus5.append(eid)
           danmu_plus6.append(eid)

                # 分类存储评论弹幕（danmakuSource = 2）

        elif j.get('danmakuSource') == 2:
                    danmu_plus2.append(eid)
        elif j.get('danmakuSource') == 3:
                    danmu_plus3.append(eid)
        elif j.get('danmakuSource') == 4:
                    danmu_plus4.append(eid)
        elif j.get('danmakuSource') == 5:
                    danmu_plus5.append(eid)
        elif j.get('danmakuSource') == 6:
                    danmu_plus6.append(eid)

# 去重并获取最终的 UID 数量、纯弹幕的 UID 数量、弹幕ID和+1ID的数量
unique_uids = len(set(all_danmu))
unique_pure_danmu_ids = len(set(pure_danmu))
unique_danmu_plus2_ids = len(set(danmu_plus2))
unique_danmu_plus3_ids = len(set(danmu_plus3))
unique_danmu_plus4_ids = len(set(danmu_plus4))
unique_danmu_plus5_ids = len(set(danmu_plus5))
unique_danmu_plus6_ids = len(set(danmu_plus6))

print(f"{title} 的 UID 总数量: {unique_uids}")
print(f"{title} 的 纯弹幕ID 总数量: {unique_pure_danmu_ids}")
print(f"{title} 的 1+2 总数量: {unique_danmu_plus2_ids}")
print(f"{title} 的 1+3 总数量: {unique_danmu_plus3_ids}")
print(f"{title} 的 1+4 总数量: {unique_danmu_plus4_ids}")
print(f"{title} 的 1+5 总数量: {unique_danmu_plus5_ids}")
print(f"{title} 的 1+6 总数量: {unique_danmu_plus6_ids}")





https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1972503008432357537&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1975132550619725955&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1977715801649578080&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1980245223854309438&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1982901424903684266&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1985499098433716288&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1988100293836931134&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/a